Getting the relevant data to code basis can be the bottleneck, as opposed to the actual code itself.

When this is the case, the program is calles I/O bound -> the speed is bounded by the efficiency of the input/output

Every time code reads from a file or writes to a network socket, it must pause to contact the kernel, request that the actual read happens, and then wait for it to complete.

This is because it's not the program but the kernel that does the actual read operation, as the kernel is responsible for managing any interaction with the hardware.



Most I/O operations on devices are multiple times slower than the CPU. So even if the communication with the kernel is fast, most time is spent for the kernel to get the result from the device and send it back to the program.

During this time the propram is halted, until it gets a signal that the write operation has completed. Time spent in a paused state is called I/O wait

Asynchronous I/O helps to utilize this waiting time by allowing  to perform other operations while the program is in I/O state:

![](./pics/IMG_6462.jpg)


This is possible because while a program is in I/O wait, the kernel is simply waiting for the device requested to read from to send a signal that the requested data is ready.


Instead of waiting, we can create a mechanism (event loop) so that we can dispatch requests of data, continue performing compute operations, and be notified when the data is ready to be read.

### Important: this is all still happening an a single thread and still uses one CPU at a time

In contrast to multiprocessing/multithreading, where a new process is launched that does experience I/O wait but uses multitasking nature of modern CPUs to allow the main process to continue.

These two mechanisms are often used in tandem: multiple processes are launched, each is efficient at asynchronous I/O.


When a programm enters I/O wait, the execution is paused so that the kernel can perform low-level operations associated with I/O request (context switch), and it is not resumed until I/O operation is completed.

Context switch is a heavy operation as it requires to save the state of the program and give up the use of the CPU.

Later the program needs to be reinitialized and resumed.

With concurrency a event loop is running that manages what gets to run in the program and when.

*Event loop = list of functions that need to be run*





In [9]:
from queue import Queue
from functools import partial
from time import sleep

eventloop = None

class EventLoop(Queue):
    def start(self):
        for i in range(4):
            function = self.get()
            function()

def do_hello():
    global eventloop
    print("Hello")
    sleep(1)
    eventloop.put(do_world)

def do_world():
    global eventloop
    print("world")
    sleep(1)
    eventloop.put(do_hello)

eventloop = EventLoop()
eventloop.put(do_hello)
eventloop.start()

Hello
world
Hello
world


The eventloop.put(do_world) approximates an asynchronous call to the do_world function.

This operation is called **nonblocking**  -> it will return immediately but guarantee that do_world is called at some point later.


## How does async/await work?

An async function (assigned with *async def*) is called coroutine and is comparable to a generator.

An await statement is similar in function to a yield statement: the execution of the current function gets paused while other code is run.

Once the await / yield resolves with data, the function is resumed. 


https://testdriven.io/blog/concurrency-parallelism-asyncio/#asyncio

RuntimeError: asyncio.run() cannot be called from a running event loop